In [ ]:
#텍스트 다중 분류 -> 각 게시글의 질문이 어떤 프로그래밍 언어에 관한 것인지 예측하는 모델 만들기 
#질문 내용(한국어)과 코드(영어)로 이루어진 다중언어 텍스트를 전처리 후 연속적인 값으로 변환하여 모델링.

#코드 텍스트를 보존하기 위해 다른 전처리는 하지 않고 Mecab으로 형태소 분리, 최소한의 한국어 불용어만 제거
#TF IDF 행렬로 벡터화 해 모델 인풋으로 사용
#각 프로그래밍 언어에서 주로 사용되는 용어와 코드를 학습하여 언어를 분류할 수 있다고 가정 
#데이터의 양이 딥러닝을 적용하기엔 충분치 않다고 판단하여 확률기반 모델인 나이브베이즈 다중분류모델 사용

In [3]:
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv(r'../프로그래머스/hashcode_classification2020_train_preprocessed.csv', encoding = 'utf-8')

In [11]:
del train["Unnamed: 0"]

In [12]:
train.sample(10)

,title,content,label,preprocessed,detokenized
230,무조건 중간에 오류 나는 코드,$$$프로세스에서 발생한 에러랑 상호 작용하는 파이썬 프로그램을 만들고 있는데요\n...,2,"['무조건', '중간', '오류', '코드', '프로세스', '발생', '에러', ...",무조건 중간 오류 코드 프로세스 발생 에러 랑 상호 작용 파이썬 프로그램 제 만든 ...
1078,확장 recyclerview 방법,recyclerview 에서 포지션 하나하나를 클릭할 때마다 포지션 밑에 레이아웃이...,3,"['확장', 'recyclerview', '방법', 'recyclerview', '...",확장 recyclerview 방법 recyclerview 포지션 하나하나 클릭 마다...
1596,안녕하세요. 디자이넌데 자바 알못이라 질문 드립니다.,하기의 링크에서 $$$/CSS를 따와 슬라이드 배너를 적용하였습니다. \n\nhtt...,4,"['안녕', '.', '디', '자', '넌데', '자바', '못', '라', '질...",안녕 . 디 자 넌데 자바 못 라 질문 . 링크 $$$/ CSS 따와 슬라이드 배 ...
2541,"HTML, CSS, $$$ 소스 암호화 관련해서 질문드립니다.",네이버 웹 문서의 경우 크롬에서 소스 보기를 클릭하면 다음과 같이 소스 계층이 보이...,4,"['HTML', ',', 'CSS', ',', '소스', '암호', '화', '관련...","HTML , CSS , 소스 암호 화 관련 해서 질문 . 네이버 웹 문서 크롬 소스..."
736,$$$코딩,"안녕하세요,$$$ 코딩이필요해서 그런데, 소스코딩해봤는데 수정 좀해주세요 ㅜㅡㅜ\n...",3,"['코딩', '안녕', ',', '코딩', '필요', '해서', '그런데', ','...","코딩 안녕 , 코딩 필요 해서 그런데 , 소스 코딩 봤 수정 ㅜㅡㅜ 속성 클래스 주..."
2253,$$$에서 Integer클래스는 어떻게 비교하죠?,\n음.. 예를 들어서\n```\nInteger a = 4;\nif (a < 5)\...,3,"['Integer', '클래스', '어떻게', '비교', '죠', '?', '음',...",Integer 클래스 어떻게 비교 죠 ? 음 . . 예 어서 ``` Integer ...
884,$$$ 스택 문자열 거꾸로 출력 질문합니다,```\n#include < stdio.h>\n#include < stdlib....,1,"['스택', '문자열', '거꾸로', '출력', '질문', '```', '#', '...",스택 문자열 거꾸로 출력 질문 ``` # include < stdio . h > #...
2479,$$$ 앱에서 데이터를 받고 저장하기,```\nfrom $$$.shortcuts import render\n\nfrom ...,5,"['앱', '데이터', '저장', '```', 'from', '$$$.', 'sho...",앱 데이터 저장 ``` from $$$. shortcuts import render...
1103,VSC: $$$ 개발 할떄,$$$를 이제 막 배워보려고 합니다. 기존에 IDE로 계속 VS코드를 쓰고있어 이클...,3,"['VSC', ':', '개발', '떄', '이제', '막', '배워', '려고',...",VSC : 개발 떄 이제 막 배워 려고 . 기존 IDE 계속 VS 코드 쓰 이클립스...
1821,$$$ numpy 질문좀여,```\n#_*_coding:utf-8 _*_\n\nimport numpy as n...,5,"['numpy', '질문', '여', '```', '#_*_', 'coding', ...",numpy 질문 여 ``` #_*_ coding : utf - 8 _*_ impor...


In [13]:
train.isnull().sum()

title           0
content         0
label           0
preprocessed    0
detokenized     0
dtype: int64

In [14]:
train.target = train['label']

C:\Users\NY\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [15]:
train.data = train["detokenized"]

C:\Users\NY\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [16]:
train.data

0       wait ( ) sleep ( ) 차이점 뭔 가요 발생 문제 및 실행 환경 wait...
1       초보 외부 클래스 멤버 변수 사용 질문 . ㅠㅠ 1 . 헤더 , 사용 멤버 변수 담...
2       ORM : Sequelize : 다대다 관계 쿼리 안녕 . 어떻게 다대다 관계 쿼리...
3       숫자 NaN 인지 검사 려면 어떻게 죠 ? ``` parseFloat ( ' geo...
4       2 . 7 중국어 , 특수 문자 인코딩 방법 뭔가요 ? ``` >>> dict [ ...
5       탐색기 시작 위치 변경 . 코딩 인데 def explorer ( ) : subpro...
6       웹 개발 . 실무 lombok 사용 ? 웹 개발 . 회사 사수 코트 수정 라 . ....
7       express 특정 단추 눌렀 , mysql 데이터 수정 , 수정 된 페이지 로드 ...
8       텍스트 파일 읽 와서 내림차순 혹은 오름차순 정리 파일 출력 9 김철수 72 10 ...
9       클래스 변수 대해 궁금 ``` Class Dog : name = str ( ) tr...
10      순서 정리 오류 질문 ``` / / # include < stdio . h > # ...
11      함수 변수 값 밖 빼 서 , 파일 import 려 . 안녕 . 간단히 설명 자면 ,...
12      배열 이름 이랑 배열 주소 건 죠 ? 발생 문제 및 실행 환경 배열 이름 이랑 배열...
13      배우 학생 . 현재 제 풀 문제 . [ https : / / noj . am / 4...
14      null 예외 처리 깔끔 법 발생 문제 및 실행 환경 보통 코드 짜 다 면 if (...
15      레이블 이나 텍스트 박스 DB select 값 출력 려면 어떻게 ? ``` @ Ov...
16      배열 이름 질문 . 안녕 . 공부 학생 . 배열 이름 관해서 궁금증 생겼 . int...
17      반 복문 작

In [61]:
train.data

0       wait ( ) sleep ( ) 차이점 뭔 가요 발생 문제 및 실행 환경 wait...
1       초보 외부 클래스 멤버 변수 사용 질문 . ㅠㅠ 1 . 헤더 , 사용 멤버 변수 담...
2       ORM : Sequelize : 다대다 관계 쿼리 안녕 . 어떻게 다대다 관계 쿼리...
3       숫자 NaN 인지 검사 려면 어떻게 죠 ? ``` parseFloat ( ' geo...
4       2 . 7 중국어 , 특수 문자 인코딩 방법 뭔가요 ? ``` >>> dict [ ...
5       탐색기 시작 위치 변경 . 코딩 인데 def explorer ( ) : subpro...
6       웹 개발 . 실무 lombok 사용 ? 웹 개발 . 회사 사수 코트 수정 라 . ....
7       express 특정 단추 눌렀 , mysql 데이터 수정 , 수정 된 페이지 로드 ...
8       텍스트 파일 읽 와서 내림차순 혹은 오름차순 정리 파일 출력 9 김철수 72 10 ...
9       클래스 변수 대해 궁금 ``` Class Dog : name = str ( ) tr...
10      순서 정리 오류 질문 ``` / / # include < stdio . h > # ...
11      함수 변수 값 밖 빼 서 , 파일 import 려 . 안녕 . 간단히 설명 자면 ,...
12      배열 이름 이랑 배열 주소 건 죠 ? 발생 문제 및 실행 환경 배열 이름 이랑 배열...
13      배우 학생 . 현재 제 풀 문제 . [ https : / / noj . am / 4...
14      null 예외 처리 깔끔 법 발생 문제 및 실행 환경 보통 코드 짜 다 면 if (...
15      레이블 이나 텍스트 박스 DB select 값 출력 려면 어떻게 ? ``` @ Ov...
16      배열 이름 질문 . 안녕 . 공부 학생 . 배열 이름 관해서 궁금증 생겼 . int...
17      반 복문 작

In [17]:
#훈련데이터의 10%는 테스트 데이터로 사용
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.data, train.target, test_size=0.1, random_state = 0)
print('X_train 크기:', X_train.shape)
print('y_train 크기:', y_train.shape)
print('X_test 크기:', X_test.shape)
print('y_test 크기:', y_test.shape)

X_train 크기: (2332,)
y_train 크기: (2332,)
X_test 크기: (260,)
y_test 크기: (260,)


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [19]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 500

In [20]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(2332, 500)
(260, 500)


In [21]:
model = MultinomialNB()
model.fit(features_train, labels_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
print("트레인셋 정확도:",accuracy_score(labels_train, model.predict(features_train)))

트레인셋 정확도: 0.8126072041166381


In [23]:
predicted = model.predict(features_test)
print("테스트셋 정확도:", accuracy_score(labels_test, predicted))

테스트셋 정확도: 0.7692307692307693


In [25]:
#교차검증으로 정확한 정확도 구하기
from sklearn.model_selection import KFold, cross_validate, cross_val_score
kfold = KFold(n_splits = 5)

scores = cross_val_score(model, features_train, labels_train, cv = kfold)
print("훈련셋 평균 정확도", scores.mean())

scores1 = cross_val_score(model, features_test, labels_test, cv = kfold)
print("테스트셋 평균 정확도", scores1.mean())

훈련셋 평균 정확도 0.7735890672818005
훈련셋 평균 정확도 0.6153846153846155


In [24]:
predicted

array([2, 3, 5, 5, 3, 5, 3, 3, 3, 3, 3, 3, 1, 3, 4, 4, 4, 5, 3, 3, 3, 5,
       3, 3, 5, 5, 4, 4, 4, 4, 3, 5, 3, 5, 5, 4, 3, 3, 5, 3, 3, 3, 5, 5,
       3, 2, 5, 3, 3, 3, 3, 3, 5, 4, 3, 4, 3, 4, 4, 3, 1, 3, 4, 4, 2, 3,
       3, 3, 1, 3, 2, 3, 2, 4, 3, 3, 5, 5, 4, 4, 4, 3, 5, 4, 4, 4, 5, 2,
       3, 3, 5, 3, 3, 2, 4, 3, 3, 5, 2, 5, 3, 5, 4, 3, 5, 2, 3, 4, 4, 2,
       4, 3, 3, 4, 5, 3, 2, 3, 3, 5, 3, 3, 5, 4, 2, 3, 3, 4, 3, 3, 4, 1,
       4, 3, 3, 3, 3, 5, 3, 4, 3, 5, 3, 3, 3, 1, 4, 4, 4, 5, 5, 3, 5, 5,
       2, 3, 5, 4, 3, 3, 5, 4, 3, 1, 5, 3, 4, 4, 2, 5, 3, 3, 3, 3, 1, 5,
       5, 4, 3, 3, 3, 5, 2, 1, 3, 3, 4, 5, 3, 3, 4, 5, 4, 3, 1, 5, 2, 5,
       3, 1, 3, 5, 5, 3, 5, 3, 3, 2, 3, 3, 3, 4, 4, 5, 3, 4, 4, 3, 5, 3,
       1, 3, 3, 2, 3, 5, 3, 5, 3, 5, 5, 5, 5, 5, 3, 3, 3, 5, 4, 3, 3, 4,
       3, 4, 4, 3, 2, 4, 3, 4, 5, 3, 3, 3, 4, 4, 5, 3, 3, 5], dtype=int64)

In [93]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, predicted)
print(confusion)

[[ 9  2  7  1  3]
 [ 0 17  4  0  2]
 [ 2  0 86  6  4]
 [ 0  0  8 41  4]
 [ 0  0 10  7 47]]


In [94]:
from sklearn.metrics import classification_report 
report = classification_report(y_test, predicted,
                           target_names = [1,2,3,4,5],output_dict=True)
print(pd.DataFrame(report).transpose())

              f1-score  precision    recall     support
1             0.545455   0.818182  0.409091   22.000000
2             0.809524   0.894737  0.739130   23.000000
3             0.807512   0.747826  0.877551   98.000000
4             0.759259   0.745455  0.773585   53.000000
5             0.758065   0.783333  0.734375   64.000000
accuracy      0.769231   0.769231  0.769231    0.769231
macro avg     0.735963   0.797907  0.706746  260.000000
weighted avg  0.763508   0.775032  0.769231  260.000000


In [8]:
#테스트 데이터에 적용
final = pd.read_csv(r'../프로그래머스/hashcode_classification2020_test_preprocessed.csv', encoding = 'utf-8')

In [99]:
final["detokenized"]

0      2 차원 배열 ( 행렬 ) 구현 은데요 . . ``` def init _ spin ...
1      uncaught referenceerror $ is not defined 에러 해결...
2      포인터 연산 ``` int arr [ ] = { 1 , 2 , 4 , 8 , 16 ...
3      Integer String 변환 천 단위 콤마 찍 은데 을까요 ? 35634646 ...
4      상속 질문 요 ! ``` import sys from $$$. QtWidgets i...
5      멀티 쓰레드 함수 srand ( time ( NULL ) ) 충분히 다른가요 ? 흔...
6      sqlite 3 sql 문 실행 려면 어떻게 할까요 ? 예 ``` conn = sq...
7      jQuery 키보드 엔터 눌린 건 죠 ? jQuery 키보드 엔터 눌린 건 죠 ? ...
8      / 뒤 공백 문자 전하 제거 려면 어떻게 ? 제 건 다 뻔한 코드 라서 그런데 / ...
9      안드로이드 UserManager . isUserAGoat ( ) 언제 사용 겁니까 ...
10     std : : string const char * char * 바꾸 방법 ` std...
11     API web API 신호 보내 . 안녕 . 우분투 환경 flask 파이썬 공부 ....
12     특정 문자열 여러 검색 ``` def find ( ) : for n in range...
13     millisecond 단위 sleep ` POSIX ` ` sleep ( x ) `...
14     _ CrtSetBreakAlloc ( ) 메모 리릭 찾 아 집니다 ! ! _ Crt...
15     Rails 파싱 질문 ! 안녕 ! nokogiri gem 이용 해서 크롤링 , 변수...
16     react 오류 질문 . 이렇게 코드 짰 ``` ``` < html > < head...
17     메모장 졌 메모장 아무것 적혀 안녕 입 문자

In [100]:
features_final = tfidf.transform(final["detokenized"]).toarray()

In [101]:
predicted_1 = model.predict(features_final)

In [102]:
predicted_1

array([5, 4, 1, 3, 5, 5, 5, 3, 5, 3, 2, 5, 5, 5, 5, 3, 4, 1, 1, 3, 3, 5,
       3, 5, 1, 5, 5, 2, 5, 3, 4, 3, 5, 4, 3, 5, 1, 4, 4, 5, 1, 4, 1, 3,
       4, 1, 3, 2, 4, 3, 4, 4, 1, 1, 1, 2, 5, 5, 3, 5, 3, 2, 5, 3, 2, 5,
       3, 5, 3, 3, 3, 5, 3, 4, 4, 4, 3, 5, 2, 3, 4, 1, 5, 4, 4, 4, 4, 5,
       2, 5, 3, 1, 3, 2, 1, 1, 1, 5, 5, 4, 3, 3, 5, 1, 2, 1, 4, 3, 1, 4,
       3, 2, 4, 3, 3, 5, 4, 4, 5, 2, 2, 2, 2, 1, 4, 3, 5, 1, 2, 5, 2, 2,
       2, 5, 4, 3, 5, 3, 3, 3, 5, 5, 3, 5, 3, 3, 3, 4, 3, 5, 4, 3, 1, 3,
       3, 5, 2, 4, 3, 4, 3, 3, 5, 4, 1, 3, 3, 4, 3, 3, 1, 3, 5, 5, 3, 5,
       4, 1, 4, 4, 3, 5, 3, 5, 2, 4, 5, 1, 3, 3, 1, 4, 5, 3, 2, 4, 4, 4,
       1, 4, 2, 3, 3, 5, 3, 2, 2, 4, 3, 4, 1, 1, 2, 3, 1, 5, 1, 3, 3, 4,
       5, 3, 4, 2, 3, 1, 5, 5, 3, 4, 5, 3, 4, 1, 2, 4, 5, 1, 5, 2, 4, 1,
       5, 2, 1, 4, 4, 1, 1, 2, 2, 3, 1, 2, 4, 4, 3, 5, 3, 2, 5, 5, 4, 1,
       5, 1, 3, 3, 3, 3, 3, 5, 3, 4, 4, 1, 2, 4, 1, 1, 2, 5, 1, 1, 5, 1,
       1, 3, 5, 3, 1, 2, 5, 1, 4, 4, 3, 3, 5, 3, 1,

In [103]:
df = pd.DataFrame(predicted_1, columns = ['label'])

In [104]:
df.to_csv('../프로그래머스/hashcode_classification2020_submission_1-3.csv', header = True, index = False)

In [ ]:
#공개점수 76